In [1]:
%matplotlib inline
%config InlineBackend.figure_format='svg'
from IPython.display import display,HTML
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from prettypandas import PrettyPandas
sns.set_style("ticks")
sns.set_context(context="notebook",font_scale=1)

In [2]:
d=pd.read_csv("profiles.csv")
print("The dataset contains {} records".format(len(d)))

m=d[d["sex"]=="m"] # male users
f=d[d["sex"]=="f"] # female users
print("{} males ({:.1%}), {} females ({:.1%})".format(
    len(m),len(m)/len(d),
    len(f),len(f)/len(d)))

PrettyPandas(d                                   # Prettyprints pandas dataframes
    .head(10)                                    # Sample the first 10 rows
    [[c for c in d.columns if "essay" not in c]] # Ignore columns with "essay" in the name (they are long)
)

The dataset contains 59946 records
35829 males (59.8%), 24117 females (40.2%)


,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn’t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,white,70,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn’t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (poorly)",single
2,38,thin,anything,socially,nan,graduated from masters program,nan,68,-1,nan,2012-06-27-09-10,"san francisco, california",nan,straight,has cats,nan,m,pisces but it doesn’t matter,no,"english, french, c++",available
3,23,thin,vegetarian,socially,nan,working on college/university,white,71,20000,student,2012-06-28-14-22,"berkeley, california",doesn’t want kids,straight,likes cats,nan,m,pisces,no,"english, german (poorly)",single
4,29,athletic,nan,socially,never,graduated from college/university,"asian, black, other",66,-1,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",nan,straight,likes dogs and likes cats,nan,m,aquarius,no,english,single
5,29,average,mostly anything,socially,nan,graduated from college/university,white,67,-1,computer / hardware / software,2012-06-29-19-18,"san francisco, california","doesn’t have kids, but might want them",straight,likes cats,atheism,m,taurus,no,"english (fluently), chinese (okay)",single
6,32,fit,strictly anything,socially,never,graduated from college/university,"white, other",65,-1,nan,2012-06-25-20-45,"san francisco, california",nan,straight,likes dogs and likes cats,nan,f,virgo,nan,english,single
7,31,average,mostly anything,socially,never,graduated from college/university,white,65,-1,artistic / musical / writer,2012-06-29-12-30,"san francisco, california","doesn’t have kids, but wants them",straight,likes dogs and likes cats,christianity,f,sagittarius,no,"english, spanish (okay)",single
8,24,nan,strictly anything,socially,nan,graduated from college/university,white,67,-1,nan,2012-06-29-23-39,"belvedere tiburon, california",doesn’t have kids,straight,likes dogs and likes cats,christianity but not too serious about it,f,gemini but it doesn’t matter,when drinking,english,single
9,37,athletic,mostly anything,not at all,never,working on two-year college,white,65,-1,student,2012-06-28-21-08,"san mateo, california",nan,straight,likes dogs and likes cats,atheism and laughing about it,m,cancer but it doesn’t matter,no,english (fluently),single


In [11]:
# Changed all -1 values in income to 0
df = d.replace({-1: 0})
PrettyPandas(df                                   # Prettyprints pandas dataframes
    .head(10)                                    # Sample the first 10 rows
    [[c for c in d.columns if "essay" not in c]] # Ignore columns with "essay" in the name (they are long)
)

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75,0,transportation,2012-06-28-20-30,"south san francisco, california","doesn’t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,white,70,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn’t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (poorly)",single
2,38,thin,anything,socially,nan,graduated from masters program,nan,68,0,nan,2012-06-27-09-10,"san francisco, california",nan,straight,has cats,nan,m,pisces but it doesn’t matter,no,"english, french, c++",available
3,23,thin,vegetarian,socially,nan,working on college/university,white,71,20000,student,2012-06-28-14-22,"berkeley, california",doesn’t want kids,straight,likes cats,nan,m,pisces,no,"english, german (poorly)",single
4,29,athletic,nan,socially,never,graduated from college/university,"asian, black, other",66,0,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",nan,straight,likes dogs and likes cats,nan,m,aquarius,no,english,single
5,29,average,mostly anything,socially,nan,graduated from college/university,white,67,0,computer / hardware / software,2012-06-29-19-18,"san francisco, california","doesn’t have kids, but might want them",straight,likes cats,atheism,m,taurus,no,"english (fluently), chinese (okay)",single
6,32,fit,strictly anything,socially,never,graduated from college/university,"white, other",65,0,nan,2012-06-25-20-45,"san francisco, california",nan,straight,likes dogs and likes cats,nan,f,virgo,nan,english,single
7,31,average,mostly anything,socially,never,graduated from college/university,white,65,0,artistic / musical / writer,2012-06-29-12-30,"san francisco, california","doesn’t have kids, but wants them",straight,likes dogs and likes cats,christianity,f,sagittarius,no,"english, spanish (okay)",single
8,24,nan,strictly anything,socially,nan,graduated from college/university,white,67,0,nan,2012-06-29-23-39,"belvedere tiburon, california",doesn’t have kids,straight,likes dogs and likes cats,christianity but not too serious about it,f,gemini but it doesn’t matter,when drinking,english,single
9,37,athletic,mostly anything,not at all,never,working on two-year college,white,65,0,student,2012-06-28-21-08,"san mateo, california",nan,straight,likes dogs and likes cats,atheism and laughing about it,m,cancer but it doesn’t matter,no,english (fluently),single


In [6]:
PrettyPandas(new                                 # Prettyprints pandas dataframes
    .head(10)                                    # Sample the first 10 rows
    [[c for c in d.columns if "essay" not in c]] # Ignore columns with "essay" in the name (they are long)
)

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75,,transportation,2012-06-28-20-30,"south san francisco, california","doesn’t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,white,70,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn’t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (poorly)",single
2,38,thin,anything,socially,nan,graduated from masters program,nan,68,,nan,2012-06-27-09-10,"san francisco, california",nan,straight,has cats,nan,m,pisces but it doesn’t matter,no,"english, french, c++",available
3,23,thin,vegetarian,socially,nan,working on college/university,white,71,20000,student,2012-06-28-14-22,"berkeley, california",doesn’t want kids,straight,likes cats,nan,m,pisces,no,"english, german (poorly)",single
4,29,athletic,nan,socially,never,graduated from college/university,"asian, black, other",66,,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",nan,straight,likes dogs and likes cats,nan,m,aquarius,no,english,single
5,29,average,mostly anything,socially,nan,graduated from college/university,white,67,,computer / hardware / software,2012-06-29-19-18,"san francisco, california","doesn’t have kids, but might want them",straight,likes cats,atheism,m,taurus,no,"english (fluently), chinese (okay)",single
6,32,fit,strictly anything,socially,never,graduated from college/university,"white, other",65,,nan,2012-06-25-20-45,"san francisco, california",nan,straight,likes dogs and likes cats,nan,f,virgo,nan,english,single
7,31,average,mostly anything,socially,never,graduated from college/university,white,65,,artistic / musical / writer,2012-06-29-12-30,"san francisco, california","doesn’t have kids, but wants them",straight,likes dogs and likes cats,christianity,f,sagittarius,no,"english, spanish (okay)",single
8,24,nan,strictly anything,socially,nan,graduated from college/university,white,67,,nan,2012-06-29-23-39,"belvedere tiburon, california",doesn’t have kids,straight,likes dogs and likes cats,christianity but not too serious about it,f,gemini but it doesn’t matter,when drinking,english,single
9,37,athletic,mostly anything,not at all,never,working on two-year college,white,65,,student,2012-06-28-21-08,"san mateo, california",nan,straight,likes dogs and likes cats,atheism and laughing about it,m,cancer but it doesn’t matter,no,english (fluently),single


In [8]:
#%% Investigate heights vs sex vs age
g=df.groupby(["sex","age"])["income"].mean()
fig,(ax1,ax2)=plt.subplots(ncols=2,sharex=True,figsize=(10,3))
ax1.plot(g["m"],color="b")
ax1.set_xlim(18,50)
ax1.set_ylim(0,100000)
ax1.set(title="Average income vs age for males",
        ylabel="income",
        xlabel="age")
ax2.plot(g["f"],color="r")
ax2.set_xlim(18,50)
ax2.set_ylim(0,100000)
ax2.set(title="Average income vs age for females",
        ylabel="income",
        xlabel="age");
for ax in (ax1,ax2):
    sns.despine(ax=ax)
fig.tight_layout()

DataError: No numeric types to aggregate

In [ ]:
fig,(ax1,ax2) = plt.subplots(ncols=2,figsize=(10,3),sharey=True,sharex=True)
sns.distplot(m["income"], ax=ax1,
             bins=range(20000,1000000),
             kde=False,
             color="b")
ax1.set_title("income distribution for males")
sns.distplot(f["income"], ax=ax2,
             bins=range(20000,1000000),
             kde=False,
             color="r")
ax2.set_title("income distribution for females")
ax1.set_ylabel("Number of users in income group")
for ax in (ax1,ax2):
    sns.despine(ax=ax)
fig.tight_layout()

/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# New Assignments for Team
 - Jarvis to focus on lat/lng & plotting geolocation (Geography)
 - Ryan to focus on jobs vs. drugs, drinks, smoking, sexual orientation (Substance Use)
 - Tati to focus on jobs vs. age, income, education (Economics)
 - Cephra to focus on religion, astrological sign, offspring (Spiritual)

#age	body_type	diet	drinks	drugs	education	ethnicity	height	income	job	last_online	location	offspring	orientation	pets	religion	sex	sign	smokes	speaks	status


#age vs drinks vs drug
#age vs religion
#location vs sign
#age vs smoke
#height vs income
#income vs drinks vs drug
#income vs education
#body_type vs location
#ethnicity vs job or income
    


